In [ ]:
import pandas as pd
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('./data/1.xlsx')
df 

In [ ]:
# 整数规划
n = df.shape[0]
xij = cp.Variable((n,4), integer = True)
tj = cp.Variable(4, integer=True)
WorkFrac = cp.Variable(n)

TotalWorkDays = df.iloc[:, 1].values
WorkDays = df.iloc[:, 2].values + 1
car = df.iloc[:,3].values
cars = np.repeat(car.reshape(-1,1), 4, axis=1)
Exps = df.iloc[:, 5:9].values
Accident = df.iloc[:,4].values
HighAccident = np.zeros(n)
LowAccident = np.zeros(n)
HighAccident[Accident >= 3] = 1
LowAccident[Accident <= 1] = 1
HighAccident = HighAccident.reshape(1,-1)
LowAccident = LowAccident.reshape(1,-1)
# print(HighAccident, LowAccident, sep='\n')
# print( cp.matmul(HighAccident, xij) - cp.matmul(LowAccident, xij))


AcciNote = cp.Variable(4, integer = True)
# Create Two Constraints
constrains = []
# 0-1变量约束 目标3 事故率尽可能对冲
constrains.append( AcciNote >= 0)
# constrains.append( AcciNote <= 1)
constrains.append( cp.matmul(HighAccident, xij) - cp.matmul(LowAccident, xij) <= cp.reshape(AcciNote, (1,4)) )
# AcciNote 在 >= 0中可以尽量的小 当且仅当能足够对冲的时候才能取到0，只要我们最小化AcciNote就能确保AcciNote倾向于0
 
constrains.append( xij >= np.ones((n,4)) * 0 )
constrains.append( xij <= np.ones((n,4)) )

constrains.append( cp.sum(xij, axis=0) >= np.ones(4) * 2 ) # 条件1
constrains.append( cp.sum(cp.multiply(xij, cars), axis=0) >= np.ones(4) ) # 条件2
constrains.append( cp.sum(xij, axis=1) <= np.ones(n) ) # 条件3

constrains.append( tj >= np.ones(4) * 0 ) # 松弛0-1变量tj
constrains.append( tj <= np.ones(4) )
constrains.append( cp.sum(cp.multiply(xij, Exps), axis=0) >= tj ) # 目标1


ei_bar = cp.sum( (Exps + xij) / 4, axis=1) # 目标2
# cp.sum([cp.sum((xij[:,i] - ei_bar)**2) for i in range(4)]) 

# cp.sum(AcciNote) # 目标3
# WorkLoad = (1+ WorkDays + cp.sum(xij, axis=1)) / (1 + TotalWorkDays + cp.sum(xij, axis=1)) 
# constrains.append( (WorkDays + cp.sum(xij, axis=1)) == WorkFrac )  
constrains.append( WorkDays + cp.sum(xij, axis=1) == cp.multiply(WorkFrac, TotalWorkDays )  )
# cp.sum( (WorkLoad - cp.sum(WorkLoad) / n)**2 )
WorkFracMean = cp.sum(WorkFrac) / n

# object
# obj = cp.Minimize( -cp.sum(tj) + cp.sum(AcciNote) )
# obj = cp.Minimize( -cp.sum(tj) * 10 + cp.sum([cp.sum((xij[:,i] - ei_bar)**2) for i in range(4)]) + 10 * cp.sum(AcciNote)  )
# obj = cp.Minimize( -cp.sum(tj) * 10 + cp.sum([cp.sum((xij[:,i] - ei_bar)**2) for i in range(4)]) + 10 * cp.sum(AcciNote) + cp.sum(WorkFrac) )
obj = cp.Minimize( -cp.sum(tj) * 10 + cp.sum([cp.sum((xij[:,i] - ei_bar)**2) for i in range(4)]) + 10 * cp.sum(AcciNote) + 100 * cp.sum( (WorkFrac - cp.sum(WorkFrac)/n)**2 ) )


In [ ]:
# solve
prob = cp.Problem(obj, constrains)
prob.solve(solver=cp.CPLEX)


# print
xijres = np.zeros((n,4))
xijres[np.abs(xij.value - 1) < 1e-5] = 1
print('Opt Target =', prob.value)
print('xij res =')
# print(xij.value)
print(xijres)
print('Targ1 : tj =', tj.value, ' —— 1为满足了')
print('Targ2 : ', np.sum([np.sum((xij[:,i].value - ei_bar.value)**2) for i in range(4)]))
print('Targ3 : AcciNote = ', AcciNote.value)
print('Targ4 : ', np.sum( (WorkFrac.value - np.sum(WorkFrac.value) / n ) ** 2))